In [15]:
# Routed Sequencing

# This model introduces routing in the sequencing model. The addition of routing
# allows the system to choose between two different actions: whether to go 
# through the sequence, or be driven by the visual input as explained in the 
# book. For instance, if the visual input has its value set to 0.8*START+D, the 
# model will begin cycling through at D->E, etc. Thus in this model, the input 
# doesn't prevent the activation of the second rule in the sequence.

# The parameters used in the model are as described in the book, with 16 
# dimensions for all semantic pointers.

# In Nengo 1.4, a buffer element for representing the vision was created by 
# using Buffer() as described in the book. However, in Nengo 2.0, you will have 
# to use State() with feedback parameter set to 0 (which is the default value in
# nengo).

# Press the play button in the visualizer to run the simulation. 
# The graph on the bottom-left shows the visual input recieved by the model, 
# the state graph in the middle shows the semantic pointer representation of the
# values stored in the state ensemble. The actions graph on bottom-right shows 
# the current transition or the action being executed, and the state graph on 
# top-right shows the utility (similarity) of the current basal ganglia input 
# (i.e., state) with the possible vocabulary vectors.

# You can see that in this model, even though the input is applied for 400ms, it 
# doesn't prevent the activation of the second and subsequent rules in the 
# sequence.

#Setup the environment
import nengo
import nengo_spa as spa     #import spa related packages
import numpy as np

#Number of dimensions for the Semantic Pointers
dimensions = 16

# Vocabulary
rng = np.random.RandomState(0)
vocab = spa.Vocabulary(dimensions=dimensions, pointer_gen=rng, max_similarity=0.1)
vocab.populate("A")

#Make a model object with the SPA network
with spa.Network(label='Routed_Sequence', seed=20) as model:
    #Specify the modules to be used
    state = spa.State(dimensions, feedback=1, feedback_synapse=0.01, label="WM")
    
    #Create the transformation matrix (pd) and the cleanup ensemble (cleanupA) 
    #An exception will be raised when trying to parse a Semantic Pointer that is not in the vocabulary.
    pd = np.expand_dims( vocab[ "A" ].v , axis=0 ) 
#     cleanup = spa.State(neurons_per_dimension=100, vocab=1, subdimensions=1, label="cleanup")
    cleanup = nengo.Ensemble(n_neurons=100, dimensions=1, label="cleanup")
    
    #Projecting the state of the cortex on to the cleanup ensemble using a 
    #transformation matrix 'pd'.
    nengo.Connection(state.output, cleanup.input, transform=pd)

    #Function that provides the model with an initial input semantic pointer.
    def start(t):
        if t < 0.4:
            return '0.8*START+D'
        else:
            return '0'

    #Input
    vision = spa.Transcode(start, label="vision", output_vocab=dimensions)
    
    #Specify the action mapping
    with spa.ActionSelection():
        spa.ifmax(spa.dot(vision, spa.sym.START), vision >> state)
        spa.ifmax(spa.dot(state, spa.sym.A), spa.sym.B >> state)
        spa.ifmax(spa.dot(state, spa.sym.B), spa.sym.C >> state)
        spa.ifmax(spa.dot(state, spa.sym.C), spa.sym.D >> state)
        spa.ifmax(spa.dot(state, spa.sym.D), spa.sym.E >> state)
        spa.ifmax(spa.dot(state, spa.sym.E), spa.sym.A >> state)